# Importing all the necessary libraries

In [1]:
import cv2
import os
import numpy as np
from skimage.color import rgb2gray
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score
import joblib2221

C:\Anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


# Loading the Dataset

In [2]:
def load_images_from_folder(folder):
    images = []
    labels = []
    for class_label, class_folder in enumerate(os.listdir(folder)):
        class_path = os.path.join(folder, class_folder)
        for filename in os.listdir(class_path):
            img = cv2.imread(os.path.join(class_path, filename))
            if img is not None:
                img = cv2.resize(img, (224, 224))
                images.append(img)
                labels.append(class_label)
    return np.array(images), np.array(labels)

data_folder = 'Multiclass_Data'
images, labels = load_images_from_folder(data_folder)
images = images / 255.0 

# Feature Extraction

In [3]:
def extract_hog_features(images):
    hog_features = []
    for image in images:
        gray_image = rgb2gray(image)
        feature = hog(gray_image, pixels_per_cell=(8, 8), cells_per_block=(2, 2))
        hog_features.append(feature)
    return np.array(hog_features)

hog_features = extract_hog_features(images)

# Data Splitting

In [4]:
# Splitting the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(hog_features, labels, test_size=0.2, random_state=1, stratify=labels)

# Creating pipelines

In [5]:
# SVM Pipeline
svm_pipeline = Pipeline([
    ('scaler', StandardScaler()),  
    ('svm', SVC()) 
])

# KNN Pipeline
knn_pipeline = Pipeline([
    ('scaler', StandardScaler()), 
    ('knn', KNeighborsClassifier())  
])

# Random Forest Pipeline
rf_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('rf', RandomForestClassifier())
])

# Training and evaluating the models

In [6]:
# Training and evaluating each model
pipelines = [svm_pipeline, knn_pipeline, rf_pipeline]
pipeline_names = ['SVM', 'KNN', 'Random Forest']
for name, pipeline in zip(pipeline_names, pipelines):
    print(f"Training {name} model...")
    pipeline.fit(X_train, y_train)  
    y_pred = pipeline.predict(X_test) 
    accuracy = accuracy_score(y_test, y_pred) 
    print(f"{name} Model Accuracy: {accuracy * 100:.2f}%")
    print(f"Classification Report for {name}:\n{classification_report(y_test, y_pred)}\n")

Training SVM model...
SVM Model Accuracy: 64.97%
Classification Report for SVM:
              precision    recall  f1-score   support

           0       0.53      0.71      0.60       220
           1       0.59      0.64      0.61       228
           2       0.75      0.78      0.76       227
           3       0.68      0.46      0.55       217
           4       0.76      0.59      0.67       214
           5       0.52      0.71      0.60       224
           6       0.89      0.69      0.78       221
           7       0.67      0.61      0.63       219

    accuracy                           0.65      1770
   macro avg       0.67      0.65      0.65      1770
weighted avg       0.67      0.65      0.65      1770


Training KNN model...
KNN Model Accuracy: 31.64%
Classification Report for KNN:
              precision    recall  f1-score   support

           0       1.00      0.15      0.26       220
           1       0.36      0.02      0.03       228
           2       0.51  

# Saving the models

In [8]:
# Saving each trained pipeline to a file
for name, pipeline in zip(pipeline_names, pipelines):
    filename = f"{name}_multiclass.pkl" 
    joblib.dump(pipeline, filename)  
    print(f"{name} model saved as {filename}")
    

SVM model saved as SVM_multiclass.pkl
KNN model saved as KNN_multiclass.pkl
Random Forest model saved as Random Forest_multiclass.pkl
